### 2. Programación Lineal con HiGHS

La programación lineal es útil para problemas como el flujo DC, donde las ecuaciones son lineales.

#### Ejemplo: Despacho Económico Simple

Modelaremos un despacho económico para minimizar el costo de generación en un sistema con 3 nodos.

### Función Objetivo
Minimizar el costo total de generación:

$$min \sum_{i \in \mathcal{N}}{C_i \cdot P_{G_i}}$$

**Restricciónes**

1. **Balance de potencia**: La generación total debe igualar la demanda total

$$\sum_{i \in \mathcal{N}} P_{G_i} = \sum_{i \in \mathcal{N}} P_{D_i}$$

2. **No negatividad**: Potencia generada no puede ser negativa

$$P_{G_i} >= 0; \forall i \in \mathcal{N}$$

3. **Límites de generación**: Ya implícitos en la definición de la variable

$$P_{G_i}^{min} \leq P_{G_i} \leq P_{G_i}^{max}; \forall i \in \mathcal{N}$$

**Nodos de Generación**

$$\mathcal{N} = {1, 2, 3}$$


In [2]:
using JuMP
using HiGHS
using DataFrames

# Datos
nodes = DataFrame(
    Nodo = [1, 2, 3],
    PGEN = [100.0, 150.0, 200.0],  # Límite de generación
    PLOAD = [50.0, 60.0, 70.0],   # Carga
    COSTO = [10.0, 15.0, 20.0]    # Costo por MW
)

num_n = nrow(nodes)


# Definición del modelo
modelo = Model(HiGHS.Optimizer)
set_optimizer_attribute(modelo, "output_flag", false)  # Suprime la salida de HiGHS

# Variables de decisión (@variable(modelo, nombre_variable))
@variable(modelo, PGEN[i = 1:num_n] <= nodes.PGEN[i])  # Potencias generadas

# Función objetivo: minimizar el costo total de generación (@objective(modelo, sentido, expresión_objetivo))
@objective(modelo, Min, sum(nodes.COSTO .* PGEN))

# Restricciones de balance de potencia (@constraint(modelo, nombre_restriccion, expresión_de_la_restricción))
@constraint(modelo, balance, sum(PGEN) == sum(nodes.PLOAD))
@constraint(modelo, limite, PGEN .>= 0.0)  # Generación no negativa

# Resolver el modelo
optimize!(modelo)

# Obtener resultados
println("Costo óptimo: ", objective_value(modelo))
println("Generación: ", value.(PGEN))

Costo óptimo: 2200.0
Generación: [100.0, 80.0, -0.0]
